In [ ]:
# Poner semillas para demostrar mejora?

# ResNet-34
#Duda: No entiendo como se supone que se utiliza el stride = (2,2), pues hacer esto cambia las dimensiones del tensor.
# ¿Como esperan luego que sume los dos tensores de dimension diferente? Para arreglarlo, lo único que se me ocurrió
# (asumo está mal) es que, en caso de que haya stride=(2,2) ,la suma se realice entre el resultado de la primera convolusion que altera las dimensiones, 
# y lo que ocurra luego de hacer la segunda convolusión. 

# El input original para  ResNet-34 es de (224,224,3), mientras que el de Cifar-10 es de (32,32,3). Como alteramos los parámetros apra arreglarlo?

In [36]:
import tensorflow as tf
 
# Display the version
print(tf.__version__)    

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, Add, ReLU
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

2.12.0


In [2]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [3]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

In [52]:
print(x_train[0][0])

[[0.23137255 0.24313725 0.24705882]
 [0.16862745 0.18039216 0.17647059]
 [0.19607843 0.18823529 0.16862745]
 [0.26666667 0.21176471 0.16470588]
 [0.38431373 0.28627451 0.20392157]
 [0.46666667 0.35686275 0.24705882]
 [0.54509804 0.41960784 0.29411765]
 [0.56862745 0.43137255 0.31372549]
 [0.58431373 0.45882353 0.34901961]
 [0.58431373 0.47058824 0.36470588]
 [0.51372549 0.40392157 0.30196078]
 [0.49019608 0.38823529 0.29803922]
 [0.55686275 0.45098039 0.35686275]
 [0.56470588 0.43921569 0.3372549 ]
 [0.5372549  0.41176471 0.30980392]
 [0.50588235 0.38039216 0.27843137]
 [0.5372549  0.41568627 0.30980392]
 [0.5254902  0.41568627 0.29803922]
 [0.48627451 0.38039216 0.25098039]
 [0.54509804 0.44313725 0.30588235]
 [0.54509804 0.43921569 0.29411765]
 [0.52156863 0.41176471 0.27058824]
 [0.53333333 0.41176471 0.29019608]
 [0.54509804 0.42352941 0.30196078]
 [0.59607843 0.47058824 0.34901961]
 [0.63921569 0.51372549 0.39215686]
 [0.65882353 0.53333333 0.42352941]
 [0.62352941 0.50588235 0.4 

In [ ]:
'''
Opción 1:
tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=avg,
    classes=10,
    classifier_activation='softmax'
)

https://keras.io/api/applications/resnet/
'''

In [60]:
classes = len(set(y_train))

def resblock(x, filters:int, s1:int, s2:int):

    if s1 == 2:
        conv1 = Conv2D(filters=filters, kernel_size=(3,3), strides=(s1,s1), padding="same", activation='relu')(x)
        x = conv1

    else:
        conv1 = Conv2D(filters=filters, kernel_size=(3,3), strides=(s1,s1), padding="same", activation='relu')(x)
    
    b = BatchNormalization()(conv1)
    conv2 = Conv2D(filters=filters, kernel_size=(3,3), strides=(s2,s2), padding="same", activation='relu')(b)
    add = Add()([x,conv2])
    r2 = ReLU()(add)
    b2 = BatchNormalization()(r2)

    return b2

In [61]:
i = Input(shape=x_train[0].shape)
x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu')(i)
x = MaxPooling2D((3,3), strides=(2,2))(x)

x = resblock(x, 64, 1, 1)
x = resblock(x, 64, 1, 1)
x = resblock(x, 64, 1, 1)

x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu')(x)

x = resblock(x, 64, 2, 1)
x = resblock(x, 64, 1, 1)
x = resblock(x, 64, 1, 1)
x = resblock(x, 64, 1, 1)

x = resblock(x, 128, 2, 1)
x = resblock(x, 128, 1, 1)
x = resblock(x, 128, 1, 1)
x = resblock(x, 128, 1, 1)
x = resblock(x, 128, 1, 1)
x = resblock(x, 128, 1, 1)

x = resblock(x, 256, 2, 1)
x = resblock(x, 256, 1, 1)
x = resblock(x, 256, 1, 1)

x = AveragePooling2D((7,7))
x = Dense(classes, activation='softmax') # Originalmente es 1000, pero para cifar10 dee ser 10. Añadimos igualmente una capa con 1000 neuronas?


In [ ]:
# https://github.com/ke511081177/Resnet34_keras/blob/main/resnet34.py 
# Resnet34 de alguien más, que no termino de comprender.

# https://stackoverflow.com/questions/64792460/how-to-code-a-residual-block-using-two-layers-of-a-basic-cnn-algorithm-built-wit
# Bloque residual

In [63]:
print(x_train[0].shape)

(32, 32, 3)
